In [ ]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

from dotenv import load_dotenv
# lädt die (versteckte) Datei die im selben Ordner liegt wie dieses notebook file
# die Datei muss .env benannt sein, diese Dateien sind in der Regel ausgeblendet, 
# weil Dateien mit . andeuten, dass diese im Explorer ausgeblendet werden.
# In der Datei
load_dotenv()

## Header
Wo wir unseren geheimen Schlüssel für ChatGPT mitschicken mit dem wir uns automatisch einloggen und authentifizieren (und auch unser Budget ansprechen)

In [ ]:
import os
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.environ["API_KEY"]}",
}

In [ ]:
import pprint
import requests

def call_openai(payload: dict) -> dict:
    ENDPOINT = "https://api.openai.com/v1/chat/completions"
        
    response = requests.post(ENDPOINT, headers=headers, json=payload)
    if response.status_code != 200:
        logger.warning(f"Fehlerhafte Antwort: {response.text}")
    endpoint_response = response.json()
    return endpoint_response

# RAG (Retrieval Augmented Generation)
wir schauen uns hier den AG Teil an, die Augmented Generation. 
Sprachmodelle haben einen Wissenshorizont. Das bedeutet das öffentliche Informationen 
nur bis zu einem bestimmten Punkt im Modell sind.
Führt man mit einem Modell eine Unterhaltung können potentiell zeitlich veraltete Antworten gegeben werden.

## Augmented Generation
Augmented Generation zielt darauf ab - POTENTIELL - aktuellere Daten in den Kontext des Modells hinzuzufügen.
Der Kontext ist einfach die Unterhaltung die wir mit dem Modell führen.
In der Praxis bedeutet das, man schiebt dem Modell als Teil der Unterhaltung neue Informationen "unter".
Woher die kommen und wie die dem Modell bereitgestellt werden hat etwas von einem Zaubertrick der erklärt wird....
Alle kochen hier mit Wasser :)

In [ ]:
payload = {
  # Das 4o klappt in der Regel besser für Bildanalysen
  "model": "gpt-4o-mini",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Wrestling Experte der alle Fragen über amerikanische Wrestler der 90er Jahre beantwortet."
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": """Wann ist Hulk Hogan gestorben?"""
        }
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
endpoint_response = call_openai(payload=payload)
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer

### Der Wrestler Hulk Hogan verstarb am 24.07.2025 - diese Event liegt hinter dem Horizont des Sprachmodells (Ende 2024)
Es gibt daher eine mittlerweile falsche Information als Antwort.
Man löst dieses Problem oder lindert es in dem man zusätzliche Quellen in die Unterhaltung mit dem Sprachmodell einbettet - die augmented Generation.

In [ ]:
from bs4 import BeautifulSoup
import requests
crawl_response = requests.get("https://de.wikipedia.org/wiki/Hulk_Hogan")
if crawl_response.status_code != 200:
    logger.warning(f"Website wurde nicht erfoglreich heruntergeladen - {crawl_response.text}")
    raise ValueError("Website nicht geladen")
else:
    website_text = crawl_response.text
    cleaned_text = BeautifulSoup(website_text).get_text()

In [ ]:
payload = {
  "model": "gpt-4o-mini",
  "messages": [
    {"role": "system", 
     "content": "Du bist ein Wrestling Experte der alle Fragen über amerikanische Wrestler der 90er Jahre beantwortet."
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": f"""Wann ist Hulk Hogan gestorben?
                      Berücksichige folgende Informationen {cleaned_text}
                      """
        }
      ]
    }
  ],
  "temperature": 0.5
}

In [ ]:
endpoint_response = call_openai(payload=payload)
chat_answer = endpoint_response.get('choices', [])[0].get('message', {}).get('content', "FEHLER")
chat_answer

### R - Retrieval
In der Praxis wissen wir nicht ob es neue Infos gibt. Wir wissen nicht zwangsläufig wo es sie gäbe wenn es sie gibt. 
Der Retrieval-Teil widmet sich der Suche und technischen Umsetzung der Suche, damit "aktuelle" Daten ergänzt werden können.
Der eigentlichen augmented generation Teil ist ein billiger Taschenspielertrick